<a href="https://colab.research.google.com/github/almiravelas/Zomato/blob/main/graded3_pandas_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zomato Exercise

Zomato is a restaurant aggregator website. The platform provides information, menus and user-reviews of restaurants as well as food delivery options from partner restaurants in select cities. You have been provided 3 datasets:

- zomato.csv contains restaurant information like average_cost_for_two, cuisines
- location.csv contains location of the restaurants like address, city, locality & etc
- ratings.csv contains aggregate rating for each restaurant

The different files are connected using the `id` columns which can be found in the different files. The id column is a unique identifier to represent a restaurant. You can use this to **merge** the different files.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1. Merge the zomato, location & ratings file into a signle dataframe and store it to the `merged_df` variable **(3pts)**

In [ ]:
# Load datasets from Google Drive
zomato = pd.read_csv('/content/drive/My Drive/zomato_datasets/zomato.csv')
location = pd.read_csv('/content/drive/My Drive/zomato_datasets/location.csv')
ratings = pd.read_csv('/content/drive/My Drive/zomato_datasets/ratings.csv')


In [ ]:
merged_df = pd.merge(zomato, location, on = 'id')
merged_df = pd.merge(merged_df, ratings, on = 'id')

In [ ]:
assert (6830, 35)==merged_df.shape

cols = ['id', 'address', 'city', 'city_id', 'country_id', 'latitude',
       'locality', 'locality_verbose', 'longitude', 'zipcode',
       'aggregate_rating', 'rating_color', 'rating_text', 'votes',
       'average_cost_for_two', 'book_url', 'cuisines', 'currency', 'deeplink',
       'events_url', 'featured_image', 'has_online_delivery',
       'has_table_booking', 'include_bogo_offers', 'is_book_form_web_view',
       'is_delivering_now', 'is_table_reservation_supported',
       'is_zomato_book_res', 'menu_url', 'mezzo_provider', 'name',
       'photos_url', 'price_range', 'thumb', 'url']

assert sorted(merged_df.columns) == sorted(cols)

2. Create a function that returns the average aggregate rating for each city. Expected returned value is a Series datatype which contains name of city as index and average aggregate rating as value. **(2pts)**

In [ ]:
def exercise_2(merged_df):
    city_avg_ratings = merged_df.groupby('city')['aggregate_rating'].mean()
    return city_avg_ratings

In [ ]:
res_series = exercise_2(merged_df)
assert True == np.isclose(3.302075, res_series[res_series.index == 'Pasay City'],
                          rtol=1e-05, atol=1e-08, equal_nan=False)[0]
assert True == np.isclose(1.890741, res_series[res_series.index == 'Valenzuela City'],
                          rtol=1e-05, atol=1e-08, equal_nan=False)[0]
assert True == np.isclose(1.558333, res_series[res_series.index == 'Malabon City'],
                          rtol=1e-05, atol=1e-08, equal_nan=False)[0]

3. Using the apply function of pandas convert the rating_text to a numeric equivalent. You are to introduce a new column in merged_df called `rating_numeric` which is based on the value from rating_text: **(3pts)**

Excellent = 5 <br>
Very Good = 4 <br>
Good = 3 <br>
Average = 2 <br>
Poor = 1 <br>
Not rated = 0 <br>

In [ ]:
def convert_rating_text(rating_text):
    # Define the mapping of rating text to numeric values
    mapping = {
        'Excellent': 5,
        'Very Good': 4,
        'Good': 3,
        'Average': 2,
        'Poor': 1,
        'Not rated': 0
    }
    # Return the corresponding numeric value
    return mapping.get(rating_text, 0)  # Default to 0 if not found

# Apply the conversion function to create the new column 'rating_numeric'
merged_df['rating_numeric'] = merged_df['rating_text'].apply(convert_rating_text)


In [ ]:
assert 868 == merged_df[merged_df.rating_numeric == 0].shape[0]
assert True == ('rating_numeric' in merged_df.columns)

4. Create a function which filters the merged dataframe to only contain rows coming in from a specific city. Expected returned value is a dataframe **(2pts)**

In [ ]:
def exercise_4(merged_df, city):
    filtered_df = merged_df[merged_df['city'] == city]
    return filtered_df

In [ ]:

assert 'Makati City' == exercise_4(merged_df, 'Makati City').city.unique()[0]
assert (976, 36) == exercise_4(merged_df, 'Makati City').shape

assert 'Mandaluyong City' == exercise_4(merged_df, 'Mandaluyong City').city.unique()[0]
assert (443, 36) == exercise_4(merged_df, 'Mandaluyong City').shape

assert (0, 36) == exercise_4(merged_df, 'Quezon  City').shape

5. Create a function to filter the dataframe to only contain certain columns passed in the function. Expected returned value is a dataframe **(1pt)**

In [ ]:
def exercise_5(merged_df, cols_to_filter):
   return merged_df[cols_to_filter]

In [ ]:
cols = ['name', 'cuisines', 'city', 'aggregate_rating']
assert sorted(cols) == sorted(exercise_5(merged_df, cols).columns)
assert (6830, 4) == exercise_5(merged_df, cols).shape

cols = ['average_cost_for_two', 'book_url', 'cuisines', 'currency', 'deeplink',
       'events_url', 'featured_image', 'has_online_delivery',
       'has_table_booking', 'include_bogo_offers', 'is_book_form_web_view']
assert sorted(cols) == sorted(exercise_5(merged_df, cols).columns)
assert (6830, 11) == exercise_5(merged_df, cols).shape


6.  What is the most common restaurant name in the dataset? **(2pts)** Write down your answer and show code used to come up with the answer.

In [ ]:
def most_common_restaurant_name(merged_df):
    return merged_df['name'].mode()[0]

most_common_name = most_common_restaurant_name(merged_df)
print(f"The most common restaurant name is: {most_common_name}")


The most common restaurant name is: Starbucks


7. Which restaurant has the most expensive average cost for two? **(2pts)** Write down your answer and show code used to come up with the answer.

In [ ]:
def most_expensive_restaurant(merged_df):
    return merged_df.loc[merged_df['average_cost_for_two'].idxmax()]

most_expensive = most_expensive_restaurant(merged_df)
print(f"The restaurant with the most expensive average cost for two is: {most_expensive['name']}")


The restaurant with the most expensive average cost for two is: The Tasting Room


8. How many restaurants do not have a zipcode? **(1pt)** Write down your answer and show code used to come up with the answer.

In [ ]:
def count_restaurants_without_zipcode(merged_df):
    return merged_df['zipcode'].isna().sum()

restaurants_without_zipcode = count_restaurants_without_zipcode(merged_df)
print(f"Number of restaurants without a zipcode: {restaurants_without_zipcode}")


Number of restaurants without a zipcode: 6713


9. How many restaurants in the dataset allow online delivery? **(1pt)** Write down your answer and show code used to come up with the answer.

In [ ]:
def count_restaurants_with_online_delivery(merged_df):
    return merged_df[merged_df['has_online_delivery'] == 1].shape[0]

restaurants_with_online_delivery = count_restaurants_with_online_delivery(merged_df)
print(f"Number of restaurants with online delivery: {restaurants_with_online_delivery}")


Number of restaurants with online delivery: 0


10. Which locatlity has more expensive food? Poblacion or Salcedo Village? **(3pts)** Write down your answer and show code used to come up with the answer.


In [ ]:
def compare_locality_expensive_food(merged_df):
    locality_costs = merged_df.groupby('locality')['average_cost_for_two'].mean()

    poblacion_cost = locality_costs.get('Poblacion', 0)
    salcedo_cost = locality_costs.get('Salcedo Village', 0)

    if poblacion_cost > salcedo_cost:
        return "Poblacion has more expensive food."
    elif salcedo_cost > poblacion_cost:
        return "Salcedo Village has more expensive food."
    else:
        return "Both localities have the same average cost for food."

locality_comparison = compare_locality_expensive_food(merged_df)
print(locality_comparison)


Salcedo Village has more expensive food.
